In [43]:
import os
from typing import *
from joblib import (Parallel,
                    delayed)

import numpy as np
import scipy as sc
from tqdm import tqdm

from src.Tools import portfolio_metrics, save_data_to_hdf
from qiskit_src.ansatz import QAOA, CP_QAOA, qubo_cost
from qiskit_src.tools import get_qubo, min_cost_partition

In [51]:
N=20
k=10
seed=0
alpha=0.001
layers=10
max_iter=1000

In [46]:
expected_returns, covariances = portfolio_metrics(n=N,
                                                          seed=seed)

constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=N,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)

max_cost, min_cost, min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result[
    's']
min_state_str = ''.join([str(_) for _ in min_state])
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances,
                     alpha=alpha,
                     lmbda=lmbda,
                     k=k)

In [ ]:


_available_methods_ = ['COBYLA', 'Nelder-Mead']
_method_idx_ = 0

ansatz = CP_QAOA(N_qubits=N,
                 cardinality=k,
                 layers=layers,
                 QUBO_matrix=Q,
                 QUBO_offset=offset,
                 with_z_phase=False)

# Initial guess for parameters (gamma, beta) of circuit
theta_min, theta_max = -np.pi, np.pi
N_xx_yy_angles = layers * (N - 1)
theta_i = np.random.uniform(low=theta_min, high=theta_max, size=N_xx_yy_angles)

# Use the get_cost method of the specific ansatz instance
res = sc.optimize.minimize(fun=ansatz.get_cost, x0=theta_i,
                           method=_available_methods_[_method_idx_],
                           options={'disp': False, 'maxiter': max_iter})

_dict_ = ansatz.get_state_probabilities(angles=res.x, flip_states=False)
res

In [48]:
best_state = list(_dict_.keys())[np.argmax(list(_dict_.values()))]
best_state, _dict_[best_state] 

('00000000100111111111', 0.999982465107264)

In [49]:
min_state_str

'01100101101001000111'

In [50]:
qubo_cost(min_state.astype(np.float64),Q)

-252.67025624306507

In [42]:
_dict_[min_state_str]

KeyError: '01100101101001'